In [61]:
#! pip install import-ipynb
#! pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 26.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 28.5 MB/s eta 0:00:0000:0100:01


In [63]:
import pandas as pd
import import_ipynb
from create_pswm import get_pswm
import math
import numpy as np
import seaborn as sns
from sklearn.metrics import precision_recall_curve

In [64]:
def validation_scores(matrix , df , index): #index è il numero del training set da prendere come validation
    df_val = df.query("Set == @index") #take only validation sequences

    # iterate among all sequences in the validation set. 
    
    validation_complete = [] #list that contains all the scores as well as additional informations
    validation_scores = [] #list that only contains the scores
    
    for _ , row in df_val.iterrows():

        seq = row["Sequence"] #extract the sequence from the dataframe
        pos = 0 # counter for window size tracking
        aa = 0 # index that moves along the sequence
        tmp_score = 0 #inizialise window score
        max_window_score = -math.inf , 0 , 0 , "seq" # initialize a variable that keeps in memory only the best window 
        
        end = min(90, len(seq)) # we end iterating on the sequence when we reach the 90th aa or when the sequence ends (sequence length < 90 aa)
        
        while aa < end - 14: # iteration on the sequence
            while pos <= 14: # iteration while the window is less than the fixed window size
                try: #try statement to handle aminoacids not found in the matrix
                    tmp_score += matrix[seq[aa + pos]][pos] # computing the score of the window
                except KeyError :
                    tmp_score += 0
                pos +=1 # increasing the window size counter
            if tmp_score > max_window_score[0]: # checking if the actual window has a better score than the previous one
                max_window_score = tmp_score, aa, aa + pos, seq[aa:aa + pos]
                
            aa += 1 # increase the starting point
            pos = 0 # resetting the window size

        validation_scores.append(max_window_score[0])
        validation_complete.append(max_window_score) 
    return validation_complete , validation_scores

In [37]:
def class_vector(df , index): #function that creates a class vector for all the sequences in a dataset given the index
    df_obs = df.query("Set == @index")
    class_list = []
    for _ , row in df_obs.iterrows():
        if row["Class"] == "Negative":
            class_list.append(0)
        else: #if row["Class"] == "Positive"
            class_list.append(1)
    return class_list

In [65]:
def best_threshold(validation_class , validation_scores , test_scores , test_class ): #secondo te cosa fa una funzione che si chiama best threshold? 

    # index --> index of the test set
    # validation_class --> classes observed in the validation set
    # validation_scores --> scores computed on the validation set

    #best_treshold --> best threshold according to the validation set
    
    # test_scores --> scores computed on the test set
    # test_class --> classes observed in the test set
    
    precision, recall, thresholds = precision_recall_curve(validation_class, validation_scores)
    f_score = (2*precision*recall) / (precision+recall)
    index_th = np.argmax(f_score)
    best_threshold = tresholds[index_th]
    predicted_class = [int(_ >= best_threshold) for _ in test_scores]

    #return the list that contains the predicted classes of the test_set
    return predicted_class

# FUNZIONE CHE NON è STATA TESTATA ANCORA !!!

In [ ]:
def metrics(predicted_class , test_class):
    

In [56]:
# TEST CELL : validation_scores function
if __name__ == "__main__":
    df=pd.read_csv("../Data_Preparation/train_bench.tsv", sep="\t")
    df_pos=df.query("Class=='Positive' and (Set=='1' or Set=='2' or Set=='3' or Set=='4' or Set=='5')").reset_index()
    for index,row in df_pos.iterrows():
            sequence_to_recover=row["Sequence"]
            cleavage_position=int(row["SPEnd"])
            df_pos.loc[index, "Sequence"] = sequence_to_recover[cleavage_position-13:cleavage_position+2]

    matrix = get_pswm(df_pos)
    a , b = validation_scores(matrix, df, "2")
    print(a)
    print(b)

[(21.89330616857519, 10, 25, 'LVALGTAGVEAEHVQ'), (-3.7745558620370305, 0, 15, 'MGLQTTKWPSHGAFF'), (6.233653185604011, 10, 25, 'ALAATTLAAYAPSEP'), (33.154709665817705, 14, 29, 'VLAAATPVRRQQCTT'), (-5.523917160517595, 0, 15, 'MSNQVFTTLRAATLA'), (48.37568124059787, 9, 24, 'AVSLACADSIDYSKV'), (30.43592826581368, 10, 25, 'ALVLMSLSGNSGANI'), (44.88240824953331, 17, 32, 'LAAPAPAPDAAPAAV'), (20.8222597449061, 9, 24, 'LWLGAVGVGVAELTE'), (18.222153724233618, 8, 23, 'LTVALCEAGPANNVV'), (-7.481312807256065, 0, 15, 'MNRKLLFVTLMVTML'), (4.595849927049917, 8, 23, 'LLFCLVGIVLSRPMP'), (3.991885176032802, 6, 21, 'TFLVVTIVCLDLGYT'), (-1.5496073147336729, 0, 15, 'MSMLFYTLITAFLIG'), (48.33133646076632, 9, 24, 'AALLGCTLAEEQAIF'), (-1.672737397574323, 7, 22, 'ASVAIMLIATVNAEY'), (-6.417688617055951, 0, 15, 'MPGKMVVILGASNIL'), (-5.57106994383354, 0, 15, 'MGSGRRALSAVPAVL'), (18.197264433084143, 12, 27, 'GAVWLLSSGHGEEQP'), (30.787857445458926, 14, 29, 'LMLVGAVLGSEDRSR'), (13.970889661238244, 7, 22, 'SLALALFSQRAR

In [59]:
# TEST CELL : class_vector function
if __name__ == "__main__":
    df=pd.read_csv("../Data_Preparation/train_bench.tsv", sep="\t")
    print(best_threshold(df , "4"))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 